## Imports

In [1]:
%pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [3]:
# Census Income dataset
census_income_kdd = fetch_ucirepo(id=117)

# data (as pandas dataframes)
X_c = census_income_kdd.data.features
y_c = census_income_kdd.data.targets

df_c = pd.concat([X_c, y_c], axis=1).reset_index(drop=True)

# Create a stratified sample of 40000 rows based on race distribution
X_sampled, _, y_sampled, _ = train_test_split(
    df_c.drop('ARACE', axis=1),
    df_c['ARACE'],
    train_size=40000,
    stratify=df_c['ARACE'],
    random_state=212
)

# Combine the sampled features and target back into a DataFrame
df_c = pd.concat([X_sampled, y_sampled], axis=1).reset_index(drop = True)
# df_c = df_c.drop(["HHDFMX", ], axis = 1)
df_c_m = df_c

In [4]:
df_c["ARACE"].value_counts()

,count
ARACE,
White,33553
Black,4093
Asian or Pacific Islander,1170
Other,733
Amer Indian Aleut or Eskimo,451


In [5]:
# List of numerical and categorical columns based on the provided dataset
numerical_cols = ['AAGE', 'CAPGAIN', 'GAPLOSS', 'MARSUPWRT']
categorical_cols = ['ACLSWKR', 'ADTINK', 'ADTOCC', 'AHGA', 'AHSCOL', 'AMARITL', 'AMJIND',
                    'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT',
                    'DIVVAL', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1']
categorical_cols_m = ['ACLSWKR', 'ADTINK', 'ADTOCC', 'AHGA', 'AHSCOL', 'AMJIND',
                    'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT',
                    'DIVVAL', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1']


# For numerical columns, fill with mean
df_c[numerical_cols] = df_c[numerical_cols].fillna(df_c[numerical_cols].mean())
for col in numerical_cols:
    df_c[col] = df_c[col].fillna(df_c[col].mean())
    df_c_m[col] = df_c_m[col].fillna(df_c_m[col].mean())
# For categorical columns, fill with mode
for col in categorical_cols:
    df_c[col] = df_c[col].fillna(df_c[col].mode()[0])
for col in categorical_cols_m:
    df_c_m[col] = df_c_m[col].fillna(df_c_m[col].mode()[0])

le = LabelEncoder()
income_class_mapping = dict()
race_class_mapping = dict()
marital_class_mapping = dict()
for column in df_c.columns:
    if df_c[column].dtype == object:
        if column == "ARACE":
          df_c[column] = le.fit_transform(df_c[column])
          race_class_mapping = {class_label: index for index, class_label in enumerate(le.classes_)}
        if(column == "income"):
          df_c[column] = le.fit_transform(df_c[column])
          income_class_mapping = {class_label: index for index, class_label in enumerate(le.classes_)}
for column in df_c_m.columns:
    if df_c_m[column].dtype == object:
        if(column == "income"):
          df_c_m[column] = le.fit_transform(df_c_m[column])
          income_class_mapping = {class_label: index for index, class_label in enumerate(le.classes_)}
        if(column == "AMARITL"):
          df_c_m[column] = le.fit_transform(df_c_m[column])
          marital_class_mapping = {class_label: index for index, class_label in enumerate(le.classes_)}


# 2. Standardize numerical columns
scaler = StandardScaler()
df_c_numerical_scaled = scaler.fit_transform(df_c[numerical_cols])
df_c_numerical_scaled = pd.DataFrame(df_c_numerical_scaled, columns=numerical_cols)
df_c_m_numerical_scaled = scaler.fit_transform(df_c_m[numerical_cols])
df_c_m_numerical_scaled = pd.DataFrame(df_c_m_numerical_scaled, columns=numerical_cols)

# 3. One-Hot Encode categorical columns
# Encode categorical variables
ohe = OneHotEncoder()
df_c_categorical_encoded = pd.DataFrame(ohe.fit_transform(df_c[categorical_cols]).toarray())
df_c_m_categorical_encoded = pd.DataFrame(ohe.fit_transform(df_c_m[categorical_cols]).toarray())


# 4. Concatenate the processed numerical and categorical features
df_c = pd.concat([df_c[["income", "ARACE"]], df_c_numerical_scaled.reset_index(drop=True),
                          df_c_categorical_encoded.reset_index(drop=True)], axis=1)
df_c_m = pd.concat([df_c_m[["income", "AMARITL"]], df_c_m_numerical_scaled.reset_index(drop=True),
                          df_c_m_categorical_encoded.reset_index(drop=True)], axis=1)

df_c.columns = df_c.columns.astype(str)
df_c_m.columns = df_c_m.columns.astype(str)

# Separate features and target again
X_c = df_c.drop('income', axis=1)
y_c = df_c['income']
X_c_m = df_c_m.drop('income', axis=1)
y_c_m = df_c_m['income']

# Split into train and test sets
X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(X_c, y_c, test_size=0.33, random_state=212)
X_c_m_train, X_c_m_test, y_c_m_train, y_c_m_test = train_test_split(X_c_m, y_c_m, test_size=0.33, random_state=212)

print("X_c_train shape:", X_c_train.shape)
print("X_c_test shape:", X_c_test.shape)
print("y_c_train shape:", y_c_train.shape)
print("y_c_test shape:", y_c_test.shape)
print("----------------------------------")
print("X_c_m_train shape:", X_c_m_train.shape)
print("X_c_m_test shape:", X_c_m_test.shape)
print("y_c_m_train shape:", y_c_m_train.shape)
print("y_c_m_test shape:", y_c_m_test.shape)

X_c_train shape: (26800, 1070)
X_c_test shape: (13200, 1070)
y_c_train shape: (26800,)
y_c_test shape: (13200,)
----------------------------------
X_c_m_train shape: (26800, 1070)
X_c_m_test shape: (13200, 1070)
y_c_m_train shape: (26800,)
y_c_m_test shape: (13200,)


## KNN with Custom Distance Functions

In [6]:
def custom_distance(x1, x2, X_train, means, mads, numeric_columns, ordinal_columns):
    distance = 0
    for i in range(len(x1)):
        column_name = X_train.columns[i]

        # Interval-scaled features (numeric)
        if column_name in numeric_columns:
            mean = means[column_name]
            mad = mads[column_name]

            if mad == 0:
                z1 = z2 = 0
            else:
                z1 = (x1[i] - mean) / mad
                z2 = (x2[i] - mean) / mad

            distance += abs(z1 - z2)

        # Ordinal features (numeric but ordered)
        elif column_name in ordinal_columns:
            min_value = X_train[column_name].min()
            max_value = X_train[column_name].max()

            if max_value == min_value:
                z1 = z2 = 0
            else:
                z1 = (x1[i] - min_value) / (max_value - min_value)
                z2 = (x2[i] - min_value) / (max_value - min_value)

            distance += abs(z1 - z2)

        # Nominal (categorical/string) features
        else:
            if x1[i] == x2[i]:
                distance += 0
            else:
                distance += 1

    return distance

# Wrapper function for scikit-learn's KNN to work with custom distances
class CustomKNN:
    def __init__(self, k=16):
        self.k = k
        self.knn = NearestNeighbors(n_neighbors=k, metric=self.custom_metric)

    def fit(self, X_train, y_train):
        """Store the training data"""
        self.X_train = X_train
        self.y_train = y_train

        # Calculate means and MADs for numeric columns
        self.numeric_columns = X_train.select_dtypes(include=[np.number]).columns
        self.ordinal_columns = ['satisfaction_level']  # Example of ordinal columns
        self.means = X_train[self.numeric_columns].mean()
        self.mads = X_train[self.numeric_columns].apply(lambda x: (x - x.mean()).abs().mean())

        # Fit the KNN model using the training data
        self.knn.fit(X_train, y_train)

    def custom_metric(self, x1, x2):
        """Custom distance metric passed to sklearn's KNN"""
        return custom_distance(x1, x2, self.X_train, self.means, self.mads, self.numeric_columns, self.ordinal_columns)

    def predict(self, X_test):
        """Predict using the trained KNN model"""
        return self.knn.predict(X_test)

## Compute Discrimination

In [7]:
def compute_discrimination(y_pred, X_test, sensitive_attribute, protected_values, negative_labels):
    """
    Computes discrimination in model predictions.

    Args:
        y_pred: The labels.
        X_test: The test dataset features.
        sensitive_attribute: The name of the sensitive attribute (e.g., 'race').
        protected_values: List of values of the sensitive attribute representing the protected group.
        negative_labels: Negative Labels in the dataset

    Returns:
        The discrimination score (difference in positive label probability).
    """

    # Create masks for protected and unprotected groups based on the sensitive attribute
    protected_mask = X_test[sensitive_attribute].isin(protected_values)
    unprotected_mask = ~protected_mask

    # Ensure neither group is empty
    if sum(protected_mask) == 0 or sum(unprotected_mask) == 0:
        return 0  # Avoid division by zero if either group is empty

    # Calculate the positive rate for both groups
    protected_group_positive_rate = 1 - (sum(pd.Series(y_pred[protected_mask]).isin(negative_labels)) / sum(protected_mask))
    unprotected_group_positive_rate = 1 - (sum(pd.Series(y_pred[unprotected_mask]).isin(negative_labels)) / sum(unprotected_mask))

    # Calculate the discrimination score
    discrimination = unprotected_group_positive_rate - protected_group_positive_rate

    return discrimination

In [ ]:
def calculate_t_discrimination_percentage(R, t, k, sensitive_attr, protected_value, label_attr, positive_label):
    """
    Calculates the percentage of t-discriminated instances in the dataset R.

    Parameters:
    - R: pandas DataFrame containing the dataset.
    - t: Threshold value for diff(r).
    - k: Number of nearest neighbors.
    - sensitive_attr: Name of the sensitive attribute in R.
    - protected_value: Value of the sensitive attribute that identifies the protected group.
    - label_attr: Name of the label attribute in R.
    - positive_label: Value of the label that represents the positive class (⊕).

    Returns:
    - percentage: The percentage of t-discriminated instances among the protected group.
    - num_t_discriminated: Number of t-discriminated instances.
    - total_protected: Total number of instances in the protected group.
    """
    # Split R into protected group P(R) and unprotected group U(R)
    P_R = R[R[sensitive_attr] == protected_value].reset_index(drop=True)
    U_R = R[R[sensitive_attr] != protected_value].reset_index(drop=True)

    # Feature columns (exclude sensitive attribute and label)
    feature_cols = [col for col in R.columns if col not in [sensitive_attr, label_attr]]

    # Initialize variables
    num_t_discriminated = 0
    total_protected = len(P_R)

    # Precompute NearestNeighbors models
    if len(P_R) > 1:
        nbrs_P = NearestNeighbors(n_neighbors=min(k+1, len(P_R))).fit(P_R[feature_cols])
    else:
        nbrs_P = None  # Not enough data for neighbors
    if len(U_R) >= 1:
        nbrs_U = NearestNeighbors(n_neighbors=min(k, len(U_R))).fit(U_R[feature_cols])
    else:
        nbrs_U = None  # No unprotected instances

    # Iterate over each protected instance
    for idx, r in P_R.iterrows():
        x_r = r[feature_cols].values.reshape(1, -1)
        dec_r = r[label_attr]

        # Compute p1 (protected neighbors)
        if nbrs_P and len(P_R) > 1:
            distances_P, indices_P = nbrs_P.kneighbors(x_r)
            indices_P = indices_P[0]
            # Exclude the instance itself if present
            indices_P = indices_P[indices_P != idx]
            k_P = min(k, len(indices_P))
            if k_P > 0:
                neighbor_labels_P = P_R.iloc[indices_P[:k_P]][label_attr].values
                p1 = np.sum(neighbor_labels_P == dec_r) / k_P
            else:
                p1 = 0
        else:
            p1 = 0

        # Compute p2 (unprotected neighbors)
        if nbrs_U and len(U_R) >= 1:
            distances_U, indices_U = nbrs_U.kneighbors(x_r)
            indices_U = indices_U[0]
            k_U = min(k, len(indices_U))
            neighbor_labels_U = U_R.iloc[indices_U[:k_U]][label_attr].values
            p2 = np.sum(neighbor_labels_U == dec_r) / k_U
        else:
            p2 = 0

        diff_r = p1 - p2

        # Check t-discrimination conditions
        if (dec_r == positive_label) and (diff_r >= t):
            num_t_discriminated += 1

    # Calculate percentage
    if total_protected > 0:
        percentage = (num_t_discriminated / total_protected) * 100
    else:
        percentage = 0

    return percentage, num_t_discriminated, total_protected

## Discrimination Discovery

In [8]:
def DiscoveryN(R, t, k, sensitive_attr, protected_values, label_attr, negative_labels):
    """
    Implements the DiscoveryN method for discrimination discovery.

    Parameters:
    - R: pandas DataFrame containing the dataset.
    - t: Threshold value for diff(r).
    - k: Number of nearest neighbors.
    - sensitive_attr: Name of the sensitive attribute in R.
    - protected_values: List of values of the sensitive attribute that identify the protected group.
    - label_attr: Name of the label attribute in R.
    - negative_labels: List of values of the label that represents the negative class.

    Returns:
    - L: DataFrame containing records from the protected group with 'disc' attribute.
    """
    # Split R into protected group P(R) and unprotected group U(R)
    P_R = R[R[sensitive_attr].isin(protected_values)].reset_index(drop=True)
    U_R = R[~R[sensitive_attr].isin(protected_values)].reset_index(drop=True)

    L_records = []

    # Feature columns (exclude sensitive attribute, label, and 'disc' if present)
    feature_cols = [col for col in R.columns if col not in [sensitive_attr, label_attr, 'disc']]

    # Precompute NearestNeighbors models for U(R)
    if len(U_R) >= k:
        nbrs_U = NearestNeighbors(n_neighbors=k).fit(U_R[feature_cols])
    else:
        nbrs_U = NearestNeighbors(n_neighbors=len(U_R)).fit(U_R[feature_cols])

    # Iterate over each record in P(R)
    for idx, r in P_R.iterrows():
        # Prepare P(R) excluding r
        P_R_excl_r = P_R.drop(idx).reset_index(drop=True)

        x_r = r[feature_cols].values.reshape(1, -1)

        # Neighbors in P(R)\{r}
        k_P = min(k, len(P_R_excl_r))
        if k_P > 0:
            nbrs_P = NearestNeighbors(n_neighbors=k_P).fit(P_R_excl_r[feature_cols])
            distances_P, indices_P = nbrs_P.kneighbors(x_r)
            ksetP_labels = P_R_excl_r.iloc[indices_P[0]][label_attr].values
            p1 = np.sum(ksetP_labels == r[label_attr]) / k_P
        else:
            p1 = 0

        # Neighbors in U(R)
        k_U = min(k, len(U_R))
        if k_U > 0:
            distances_U, indices_U = nbrs_U.kneighbors(x_r)
            ksetU_labels = U_R.iloc[indices_U[0]][label_attr].values
            p2 = np.sum(ksetU_labels == r[label_attr]) / k_U
        else:
            p2 = 0

        diff = p1 - p2

        # Determine 'disc' attribute based on conditions
        if (r[label_attr] in negative_labels) and diff >= t:
            r['disc'] = 'yes'
        else:
            r['disc'] = 'no'

        L_records.append(r)

    # Create DataFrame L from records
    L = pd.DataFrame(L_records)
    return L

### Race as Sensitive Attribute

In [10]:
# Parameters
t = 0.1
k = 16
sensitive_attr = 'ARACE'
protected_values = [race_class_mapping[v] for v in [' Black', ' Amer Indian Aleut or Eskimo',' Asian or Pacific Islander', ' Other']]
label_attr = 'income'
negative_labels = [income_class_mapping[v] for v in ['-50000']]

# Run DiscoveryN
L = DiscoveryN(df_c, t, k, sensitive_attr, protected_values, label_attr, negative_labels)

In [ ]:
X_L = L.drop(['disc'], axis=1)
y_L = L['disc']


# Split L into train and test sets
X_L_train, X_L_test, y_L_train, y_L_test = train_test_split(X_L, y_L, test_size=0.2, random_state=42)

# Train a DecisionTreeClassifier on L
clf = DecisionTreeClassifier()
clf.fit(X_L_train, y_L_train)

# Make predictions on the test set
y_L_pred = clf.predict(X_L_test)

# Evaluate the classifier
accuracy = accuracy_score(y_L_test, y_L_pred)
precision = precision_score(y_L_test, y_L_pred, pos_label='yes')
recall = recall_score(y_L_test, y_L_pred, pos_label='yes')
f1 = f1_score(y_L_test, y_L_pred, pos_label='yes')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.9209302325581395
Precision: 0.323943661971831
Recall: 0.2987012987012987
F1-score: 0.3108108108108108


### Marital Status as Sensitive Attribute

In [11]:
# Parameters
t = 0.1
k = 16
sensitive_attr = 'AMARITL'
protected_values = [marital_class_mapping[v] for v in [' Divorced', ' Separated', ' Widowed']]
label_attr = 'income'
negative_labels = [income_class_mapping[v] for v in ['-50000']]

# Run DiscoveryN
L = DiscoveryN(df_c_m, t, k, sensitive_attr, protected_values, label_attr, negative_labels)

In [13]:
X_L = L.drop(['disc'], axis=1)
y_L = L['disc']

# Split L into train and test sets
X_L_train, X_L_test, y_L_train, y_L_test = train_test_split(X_L, y_L, test_size=0.2, random_state=42)

# Train a DecisionTreeClassifier on L
clf = DecisionTreeClassifier()
clf.fit(X_L_train, y_L_train)

# Make predictions on the test set
y_L_pred = clf.predict(X_L_test)

# Evaluate the classifier
accuracy = accuracy_score(y_L_test, y_L_pred)
precision = precision_score(y_L_test, y_L_pred, pos_label='yes')
recall = recall_score(y_L_test, y_L_pred, pos_label='yes')
f1 = f1_score(y_L_test, y_L_pred, pos_label='yes')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Accuracy: 0.896358543417367
Precision: 0.22058823529411764
Recall: 0.2054794520547945
F1-score: 0.21276595744680848


## Discrimation Prevention

In [14]:
def PreventionN(T, V, t, k, sensitive_attr, protected_values, label_attr, negative_labels, classifierName = "DT"):
    """
    Implements the PreventionN method.

    Parameters:
    - T: pandas DataFrame representing the training dataset.
    - V: pandas DataFrame representing the validation dataset.
    - t: Threshold value for diff(r).
    - k: Number of nearest neighbors.
    - sensitive_attr: Name of the sensitive attribute in T.
    - protected_values: Value of the sensitive attribute that identifies the protected group.
    - label_attr: Name of the label attribute in T.
    - negative_labels: List of the labels that represents the negative class.

    Returns:
    - classifier_T: Classifier trained on the original training dataset T.
    - classifier_T_prime: Classifier trained on the modified training dataset T'.
    - performance: A dictionary containing accuracy scores on V for both classifiers.
    """
    # Copy of the training data for T'
    T_prime = T.copy()

    # Split T into protected and unprotected groups
    P_T = T[T[sensitive_attr].isin(protected_values)].reset_index(drop=True)
    U_T = T[~T[sensitive_attr].isin(protected_values)].reset_index(drop=True)

    # Feature columns (exclude sensitive attribute and label)
    feature_cols = [col for col in T.columns if col not in [sensitive_attr, label_attr]]

    # Precompute NearestNeighbors models
    if len(P_T) > 1:
        nbrs_P = NearestNeighbors(n_neighbors=min(k, len(P_T)-1)).fit(P_T[feature_cols])
    else:
        nbrs_P = None  # Not enough data
    if len(U_T) >= k:
        nbrs_U = NearestNeighbors(n_neighbors=k).fit(U_T[feature_cols])
    else:
        nbrs_U = NearestNeighbors(n_neighbors=len(U_T)).fit(U_T[feature_cols])

    # Iterate over each record in T
    for idx, r in T.iterrows():
        # Check if record is protected
        is_protected = r[sensitive_attr] in protected_values

        x_r = r[feature_cols].values.reshape(1, -1)
        dec_r = r[label_attr]

        # Compute diff(r) only if necessary
        if is_protected:
            # Neighbors in P_T excluding r
            if nbrs_P and len(P_T) > 1:
                P_T_excl_r = P_T.drop(P_T.index[idx if idx < len(P_T) else -1]).reset_index(drop=True)
                k_P = min(k, len(P_T_excl_r))
                if k_P > 0:
                    nbrs_P_excl = NearestNeighbors(n_neighbors=k_P).fit(P_T_excl_r[feature_cols])
                    distances_P, indices_P = nbrs_P_excl.kneighbors(x_r)
                    ksetP_labels = P_T_excl_r.iloc[indices_P[0]][label_attr].values
                    p1 = np.sum(ksetP_labels == dec_r) / k_P
                else:
                    p1 = 0
            else:
                p1 = 0

            # Neighbors in U_T
            k_U = min(k, len(U_T))
            if k_U > 0:
                distances_U, indices_U = nbrs_U.kneighbors(x_r)
                ksetU_labels = U_T.iloc[indices_U[0]][label_attr].values
                p2 = np.sum(ksetU_labels == dec_r) / k_U
            else:
                p2 = 0

            diff = p1 - p2
        else:
            diff = 0  # For unprotected, diff is not computed or used

        # Conditions to modify the decision attribute in T_prime
        if (dec_r in negative_labels) and is_protected and (diff >= t):
            T_prime.at[idx, label_attr] = income_class_mapping[" 50000+."]

    # Build classifiers on T and T_prime
    X_T = T[feature_cols]
    y_T = T[label_attr]

    X_T_prime = T_prime[feature_cols]
    y_T_prime = T_prime[label_attr]

    if(classifierName == "DT"):
      classifier_T = DecisionTreeClassifier()
    elif(classifierName == "NB"):
      classifier_T = GaussianNB()
    elif(classifierName == "LR"):
      classifier_T = LogisticRegression()
    classifier_T.fit(X_T, y_T)

    if(classifierName == "DT"):
      classifier_T_prime = DecisionTreeClassifier()
    elif(classifierName == "NB"):
      classifier_T_prime = GaussianNB()
    elif(classifierName == "LR"):
      classifier_T_prime = LogisticRegression()
    classifier_T_prime.fit(X_T_prime, y_T_prime)

    # Evaluate classifiers on validation set V
    X_V = V[feature_cols]
    y_V = V[label_attr]

    y_pred_T = classifier_T.predict(X_V)
    y_pred_T_prime = classifier_T_prime.predict(X_V)

    performance = {
        'accuracy_T': accuracy_score(y_V, y_pred_T),
        'discrimination_T': compute_discrimination(y_pred_T, pd.concat([X_V, V[["race"]]], axis = 1), sensitive_attr, protected_values, negative_labels),
        'accuracy_T_prime': accuracy_score(y_V, y_pred_T_prime),
        'discrimination_T_prime': compute_discrimination(y_pred_T_prime, pd.concat([X_V, V[["race"]]], axis = 1), sensitive_attr, protected_values, negative_labels),
    }

    return classifier_T, classifier_T_prime, performance

### Experiments

#### Race

In [15]:
# Parameters
t = 0.1
k = 16
sensitive_attr = 'ARACE'
protected_values = [race_class_mapping[v] for v in [' Black', ' Amer Indian Aleut or Eskimo',' Asian or Pacific Islander', ' Other']]
label_attr = 'income'
negative_labels = [income_class_mapping[v] for v in ['-50000']]

In [16]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "DT")
performance

{'accuracy_T': 0.9253787878787879,
 'discrimination_T': 0.040649700831287587,
 'accuracy_T_prime': 0.9170454545454545,
 'discrimination_T_prime': 0.017234623988881426}


In [17]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "NB")
performance

{'accuracy_T': 0.88825757575757575,
 'discrimination_T': 0.11710605026003873,
 'accuracy_T_prime': 0.8790151515151515,
 'discrimination_T_prime': 0.09464417153822181}


In [18]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "LR")
performance

{'accuracy_T': 0.9486363636363636,
 'discrimination_T': 0.020847829172955912,
 'accuracy_T_prime': 0.9346212121212121,
 'discrimination_T_prime': -0.017900813063304377}


In [19]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "DT")
performance

{'accuracy_T': 0.923030303030303,
 'discrimination_T': 0.019734294516318474,
 'accuracy_T_prime': 0.9073484848484848,
 'discrimination_T_prime': -0.09971345736243775}


In [20]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "NB")
performance

{'accuracy_T': 0.88825757575757575,
 'discrimination_T': 0.11710605026003873,
 'accuracy_T_prime': 0.834469696969697,
 'discrimination_T_prime': 0.075671842531305}


In [21]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_train, y_c_train], axis = 1), pd.concat([X_c_test, y_c_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "LR")
performance

{'accuracy_T': 0.9486363636363636,
 'discrimination_T': 0.020847829172955912,
 'accuracy_T_prime': 0.934469696969697,
 'discrimination_T_prime': -0.09621653661237639}


#### Marital Status

In [22]:
# Parameters
t = 0.1
k = 16
sensitive_attr = 'AMARITL'
protected_values = [marital_class_mapping[v] for v in [' Divorced', ' Separated', ' Widowed']]
label_attr = 'income'
negative_labels = [income_class_mapping[v] for v in ['-50000']]

In [23]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "DT")
performance

{'accuracy_T': 0.9246212121212121,
 'discrimination_T': 0.0307641889382847822,
 'accuracy_T_prime': 0.9113636363636364,
 'discrimination_T_prime': 0.01860417648813219}


In [24]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "NB")
performance

{'accuracy_T': 0.88825757575757575,
 'discrimination_T': -0.3098426872588048,
 'accuracy_T_prime': 0.87303030303030305,
 'discrimination_T_prime': -0.31603418530204497}


In [25]:
# 0.1
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), t, k, sensitive_attr, protected_values, label_attr, negative_labels, "LR")
performance

{'accuracy_T': 0.9486363636363636,
 'discrimination_T': 0.009884360850877183,
 'accuracy_T_prime': 0.9470454545454545,
 'discrimination_T_prime': -0.026509636628728628}


In [26]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "DT")
performance

{'accuracy_T': 0.9252272727272727,
 'discrimination_T': -0.00014435174470028844,
 'accuracy_T_prime': 0.906969696969697,
 'discrimination_T_prime': -0.15660210927619944}


In [28]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "NB")
performance

{'accuracy_T': 0.8846969696969697,
 'discrimination_T': 0.04687500231808417,
 'accuracy_T_prime': 0.82727272727272,
 'discrimination_T_prime': 0.01625218363539806}


In [27]:
# 0.05
classifier_T, classifier_T_prime, performance = PreventionN(pd.concat([X_c_m_train, y_c_m_train], axis = 1), pd.concat([X_c_m_test, y_c_m_test], axis = 1), 0.05, k, sensitive_attr, protected_values, label_attr, negative_labels, "LR")
performance

{'accuracy_T': 0.9440151515151516,
 'discrimination_T': 0.0036914781386230278,
 'accuracy_T_prime': 0.9053030303030303,
 'discrimination_T_prime': -0.1590719631772991}
